In [ ]:
!nvidia-smi

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


## import and install

In [ ]:
!pip install -q datasets
!pip install -q thai_segmenter
!pip install -q transformers
!pip install -q sentencepiece
!pip install -q huggingface_hub
!pip install -q sacremoses

In [ ]:
!huggingface-cli login

In [5]:
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
import torch
from tqdm.notebook import tqdm
import numpy as np
from typing import List, Union
import nltk
import pandas as pd

In [6]:
from datasets import load_dataset, Dataset, DatasetDict
import glob
import os

## Opus

In [10]:
class Opus:
    def __init__(self,model_name: str, device:str = None):
        self._model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(self._model_name)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(self._model_name)

        if device is None:
            device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.device = device

        self.max_length = None
    
    def translate(self, documents: Union[str, List[str]], show_progress_bar: bool = False, beam_size: int = 5, 
                        batch_size: int = 16, paragraph_split: str = "\n"):

        if isinstance(documents, str):
            translated_sent = self.model_translator(documents)
            return translated_sent[0]
        else:
            # Split document into sentences
            splitted_sentences = []
            sent2doc = []
            for doc in documents:
                paragraphs = doc.split(paragraph_split) if paragraph_split is not None else [doc]
                for para in paragraphs:
                    for sent in self._sentence_splitting(para.strip()):
                        sent = sent.strip()
                        if len(sent) > 0:
                            splitted_sentences.append(sent)
                sent2doc.append(len(splitted_sentences))

                translated_sentences = self.translate_sentences(splitted_sentences, show_progress_bar=show_progress_bar, beam_size=beam_size, batch_size=batch_size)

            # Merge sentences back to documents
            translated_docs = []
            for doc_idx in range(len(documents)):
                start_idx = sent2doc[doc_idx - 1] if doc_idx > 0 else 0
                end_idx = sent2doc[doc_idx]
                translated_docs.append(self._reconstruct_document(documents[doc_idx], splitted_sentences[start_idx:end_idx], translated_sentences[start_idx:end_idx]))

            return translated_docs

    def translate_sentences(self, sentences: Union[str, List[str]], show_progress_bar: bool = False, beam_size: int = 5, batch_size: int = 32):
        #Sort by length to speed up processing
        length_sorted_idx = np.argsort([-len(sen) for sen in sentences])
        sentences_sorted = [sentences[idx] for idx in length_sorted_idx]

        iterator = range(0, len(sentences_sorted), batch_size)
        output = []

        if show_progress_bar:
            scale = min(batch_size, len(sentences))
            iterator = tqdm.tqdm(iterator, total=len(sentences)/scale, unit_scale=scale, smoothing=0)

        for start_idx in iterator:
            output.extend(self.model_translator(sentences_sorted[start_idx:start_idx+batch_size], beam_size=beam_size))

        #Restore original sorting of sentences
        output = [output[idx] for idx in np.argsort(length_sorted_idx)]
        
        return output

    def model_translator(self, sentences:str, beam_size:int=5):
        self.model.to(self.device)

        inputs = self.tokenizer(sentences, truncation=True, padding=True, max_length=self.max_length, return_tensors="pt").input_ids
        inputs = inputs.to(self.device)

        with torch.no_grad():
            translated = self.model.generate(inputs, num_beams=beam_size)
            output = [self.tokenizer.decode(t, skip_special_tokens=True) for t in translated]
        return output

    @staticmethod
    def _sentence_splitting(text: str):
        try:
            nltk.data.find('tokenizers/punkt')
        except LookupError:
            nltk.download('punkt')
        return nltk.sent_tokenize(text)

    @staticmethod
    def _reconstruct_document(doc, org_sent, translated_sent):
          sent_idx = 0
          char_idx = 0
          translated_doc = ""
          while char_idx < len(doc):
              if sent_idx < len(org_sent) and doc[char_idx] == org_sent[sent_idx][0]:
                  translated_doc += translated_sent[sent_idx]
                  char_idx += len(org_sent[sent_idx])
                  sent_idx += 1
              else:
                  translated_doc += doc[char_idx]
                  char_idx += 1
          return translated_doc

## Translate

test translation

In [ ]:
model = Opus("Helsinki-NLP/opus-mt-en-sv")

In [14]:
documents = ["""The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building, and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest man-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure to reach a height of 300 metres. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct."""]

print(model.translate(documents))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['Tornet är 324 meter högt, ungefär lika högt som en byggnad med 81 våningar, och Paris högsta byggnad. Dess bas är kvadrat, mäter 125 meter (410 ft) på varje sida. Under sin konstruktion överträffade Eiffeltornet Washingtonmonumentet för att bli världens högsta konstgjordaste byggnad, en titel som det hade i 41 år tills Chryslerbyggnaden i New York blev färdig 1930. Det var den första byggnaden som nådde en höjd av 300 meter. På grund av att en sändningsantenn lades till på tornets ovansida 1957 är den nu högre än Chryslerbyggnaden med 5,2 meter (17 ft). Förutom sändare är Eiffeltornet den näst högsta fristående strukturen i Frankrike efter Millau Viaduct.']


### Translating hugginface_datasets

In [45]:
class NMT:
    def __init__(self, model:str, dataset_name, file_id, data_range_to_translate , data_type, skip_cols_to_translate, file_folder_drive ):
        self.opus = model
        self.dataset_name= dataset_name
        self.file_id = file_id
        self.data_range_to_translate = data_range_to_translate
        self.data_type = data_type
        self.skip_cols_to_translate = skip_cols_to_translate
        self.file_folder_drive = file_folder_drive
        self.dataset = load_dataset(self.dataset_name, split=f'{self.data_type}{self.data_range_to_translate}')
        self.path = f'/content/drive/MyDrive/{self.file_folder_drive}/df_{self.dataset_name}_{self.data_type}{str(self.file_id)}.csv'

        # print(self.dataset)

    def translate_dataset(self, batch_size:int = 64, beam_size:int=5):
        df_data_temp = []
        col_list_dataset = list(self.dataset[0].keys())

        for row_data in tqdm(self.dataset,desc='Translation in progress') :
            row_data_temp = []
            for c_list in col_list_dataset:
                to_translate_row =row_data[c_list]
                if c_list in skip_cols_to_translate:
                    is_translated_row = to_translate_row
                else:
                    is_translated_row = self.opus.translate(to_translate_row, batch_size=batch_size,beam_size=beam_size)
                row_data_temp.append(is_translated_row)
            data_zip = tuple(row_data_temp)
            df_data_temp.append(data_zip)

        return pd.DataFrame(df_data_temp, columns = col_list_dataset)

    def subdata_to_drive(self, df_trans):
      with open(self.path, 'w', encoding = 'utf-8') as f:
        df_trans.to_csv(f, sep=";")

    @staticmethod
    def assemble_df_data_type(file_folder_drive,dataset_name,data_type):
      # setting the path for joining multiple files
      files = os.path.join(f'/content/drive/MyDrive/{file_folder_drive}/', f'df_{dataset_name}_{data_type}*.csv')
      files = glob.glob(files)
      files.sort()
      df_list = [pd.read_csv(f, sep=';') for f in files]  
      df = pd.concat(df_list, ignore_index=True)    
      df_merged = df.copy()
      df_merged = df_merged.iloc[: , 1:]
      return Dataset.from_pandas(df_merged)


# Dict

Unfortunately, tune this values after your hardware.
Also, incremtnally change file_id and data_range_to_translate to assembly files afterwards (due to limitations of google colab idle).

Local station you can just hammer on without be annoyed by splitting files and keeping track of ids.. + you can cache both model and dataset..

In [46]:
dataset_name = "xsum"
file_id=1 
data_type = "train"
file_folder_drive = "xsum"
data_range_to_translate= "[0:5]"
skip_cols_to_translate=['id']

nmt = NMT( model, dataset_name, file_id, data_range_to_translate , data_type, skip_cols_to_translate, file_folder_drive)

In [47]:
df_trans= nmt.translate_dataset()
nmt.subdata_to_drive(df_trans)

Translation in progress:   0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


After translation of all files.. Assemble train, test, val

In [48]:
train_dataset = nmt.assemble_df_data_type("xsum", "xsum","train")
# test_dataset = nmt.assemble_df_data_type("xsum", "xsum","test")
# val_dataset =  nmt.assemble_df_data_type("xsum", "xsum","val")

In [ ]:
master_dataset_dict = DatasetDict({"train":train_dataset,"test":test_dataset,"validation":val_dataset})

push to hub

In [ ]:
master_dataset_dict.push_to_hub("Gabriel/xsum_swe")